In [1]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
from torchsummary import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model = models.vgg16(pretrained=True).to(device)

In [3]:
summary(model, torch.zeros(1, 3, 224, 224))

C:\Users\MSI\anaconda3\envs\tf-latest\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 7, 7]           --
|    └─Conv2d: 2-1                       [-1, 64, 224, 224]        1,792
|    └─ReLU: 2-2                         [-1, 64, 224, 224]        --
|    └─Conv2d: 2-3                       [-1, 64, 224, 224]        36,928
|    └─ReLU: 2-4                         [-1, 64, 224, 224]        --
|    └─MaxPool2d: 2-5                    [-1, 64, 112, 112]        --
|    └─Conv2d: 2-6                       [-1, 128, 112, 112]       73,856
|    └─ReLU: 2-7                         [-1, 128, 112, 112]       --
|    └─Conv2d: 2-8                       [-1, 128, 112, 112]       147,584
|    └─ReLU: 2-9                         [-1, 128, 112, 112]       --
|    └─MaxPool2d: 2-10                   [-1, 128, 56, 56]         --
|    └─Conv2d: 2-11                      [-1, 256, 56, 56]         295,168
|    └─ReLU: 2-12                        [-1, 256, 56, 56]      

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 512, 7, 7]           --
|    └─Conv2d: 2-1                       [-1, 64, 224, 224]        1,792
|    └─ReLU: 2-2                         [-1, 64, 224, 224]        --
|    └─Conv2d: 2-3                       [-1, 64, 224, 224]        36,928
|    └─ReLU: 2-4                         [-1, 64, 224, 224]        --
|    └─MaxPool2d: 2-5                    [-1, 64, 112, 112]        --
|    └─Conv2d: 2-6                       [-1, 128, 112, 112]       73,856
|    └─ReLU: 2-7                         [-1, 128, 112, 112]       --
|    └─Conv2d: 2-8                       [-1, 128, 112, 112]       147,584
|    └─ReLU: 2-9                         [-1, 128, 112, 112]       --
|    └─MaxPool2d: 2-10                   [-1, 128, 56, 56]         --
|    └─Conv2d: 2-11                      [-1, 256, 56, 56]         295,168
|    └─ReLU: 2-12                        [-1, 256, 56, 56]      

## Age and face detector

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl
import cv2
import glob 
import time 

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F 
from torchvision import transforms, models, datasets

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
trn_df = pd.read_csv(r'D:\Data Science projects\pytorch\train_labels.csv')
val_df = pd.read_csv(r'D:\Data Science projects\pytorch\val_labels.csv')

In [6]:
trn_df['age'] = trn_df['age'].astype('string')

In [7]:
splited = [i.lstrip('more than') for i in trn_df['age']] #faster to do two different list comprehension

In [8]:
splited = [i.split("-") for i in splited]

In [9]:
splited_ = []
for n in range(0, len(splited)):
    splited_.append(splited[n][0])

In [10]:
trn_df['age'] = splited_

In [11]:
#build the class

class GenderAgeclass(Dataset):
    def __init__(self, df, tfms= None):
        self.df = df
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, ix):
        f = self.df.iloc[ix].squeeze() #agarra una fila 
        file = f.file #el nombre de la columna file, obtenemos la info
        gen = f.gender == 'Female'
        age = f.age
        im = cv2.imread(file)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        return im, age, gen
    
    #Preprocess (resize, permute and normalize) the images with a function

    def preprocess_image(self, im): #the preprocess process
        im = cv2.resize(im, (IMAGE_SIZE, IMAGE_SIZE))
        im = torch.tensor(im).permute(2,0,1)
        im = self.normalize(im/255.)
        return im[None]
    
    def collate_fn(self, batch): #apply the preprocess, convert age into floats
        ims, ages, genders = [], [], []
        for im, age, gender in batch:
            im = self.preprocess_image(im) #preprocess each image
            ims.append(im)

            ages.append(float(int(age)/80))#scale age by the max  age so values are between 0 and 1
            genders.append(float(gender)) #convert it to float so they go into tensors

        ages, genders = [torch.tensor(x).to(device).float() for x in [ages, genders]]#convert to tensors for the NN

        ims = torch.cat(ims).to(device)

        return ims, ages, genders

In [12]:
#okay, now let's actually preprocess the data

trn = GenderAgeclass(trn_df)
val = GenderAgeclass(val_df)

In [13]:
#Now, charge the Dataloaders as ammo for the NN
train_loader= DataLoader(trn, batch_size=32, shuffle = True, drop_last= True, collate_fn= trn.collate_fn)
test_loader = DataLoader(val, batch_size=32, shuffle=True, drop_last=True ,collate_fn= val.collate_fn)

In [14]:
def get_model():
    model = models.vgg16(pretrained = True)
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.Sequential(
        nn.Conv2d(512,512, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten()
    )
    ### Now comes the key part. We deviate from what we have learned so far by creating two branches of outputs. 
    class ageGenderClassifier(nn.Module):
        def __init__(self):
            super(ageGenderClassifier, self).__init__()
             #define the intermediate layer calculations
            self.intermediate = nn.Sequential(
                nn.Linear(2048,512),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(512,128),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(128,64),
                nn.ReLU(),
            )
            self.age_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
            self.gender_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        #Define the forward pass method that stacks layers as intermediate
        #first, followed by age_classifier and then gender_classifier:
        def forward(self, x):
            x = self.intermediate(x)
            age = self.age_classifier(x)
            gender = self.gender_classifier(x)
            return gender, age#returns the classification and the age
    
    #define the loss functionn, optimizer and put the model on GPU
        
    model.classifier = ageGenderClassifier()
    
    gender_criterion = nn.BCELoss()
    age_criterion = nn.L1Loss()
    loss_functions = gender_criterion, age_criterion
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-4)
    
    return model.to(device), loss_functions, optimizer

model, loss_functions, optimizer = get_model()

In [15]:
#Define the train on batch of data function

# The train_batch method takes an image, actual values of gender, age,model, optimizer, and loss function, 
# as input to calculate total loss, as follows

def train_batch(data, model, optimizer, criteria):
    
    model.train()
    ims, age, gender = data
    optimizer.zero_grad()#we do this first for this part
    pred_gender, pred_age = model(ims)
    gender_criterion, age_criterion = criteria #Fetch the loss functions for both age and gender before calculating the loss 
    gender_loss = gender_criterion(pred_gender.squeeze(), gender)
    age_loss = age_criterion(pred_age.squeeze(), age)
    total_loss = gender_loss+age_loss #Calculate the overall loss by summing up gender_loss and age_loss and perform backpropagation to reduce the overall loss 
    total_loss.backward()
    optimizer.step()
    return total_loss 

In [16]:
#Now, let's do the previous part for the validation batch

def validate_batch(data, model, criteria):
    model.eval()
    with torch.no_grad():
        pred_gender, pred_age = model(img) # no gradient calculations are required before predicting the age and gender value
  #Calculate the overall loss, final predicted gender class (pred_gender), and return the predicted gender, age, and total loss:  
        gender_criterion, age_criterion = criteria
        gender_loss = gender_criterion(pred_gender.squeeze(), gender)
        age_loss = age_criterion(pred_age.squeeze(), age)
  #Calculate the overall loss, final predicted gender class (pred_gender), and return the predicted gender, age, and total loss:
        total_loss = gender_loss + age_loss
        pred_gender = (pred_gender > 0.5).squeeze()
        #gender accuracy
        gender_acc = (pred_gender == gender).float().sum()
        #age prediction
        age_mae = torch.abs(age-pred_age).float().sum()
        return total_loss, gender_acc, age_mae

In [17]:
IMAGE_SIZE = 224

In [18]:
model, criteria, optimizer = get_model()
val_gender_accuracies = []
val_age_maes = []
train_losses = []
val_losses = []

n_epochs = 5
best_test_loss = 1000
start = time.time()

for epoch in range(n_epochs):
    epoch_train_loss, epoch_test_loss = 0, 0
    val_age_mae, val_gender_acc, ctr = 0, 0, 0
    _n = len(train_loader)
    for ix, data in enumerate(train_loader):
        # if ix == 100: break
        loss = train_batch(data, model, optimizer, criteria)
        epoch_train_loss += loss.item()

    for ix, data in enumerate(test_loader):
        # if ix == 10: break
        loss, gender_acc, age_mae = validate_batch(data, model, criteria)
        epoch_test_loss += loss.item()
        val_age_mae += age_mae
        val_gender_acc += gender_acc
        ctr += len(data[0])

    val_age_mae /= ctr
    val_gender_acc /= ctr
    epoch_train_loss /= len(train_loader)
    epoch_test_loss /= len(test_loader)

    elapsed = time.time()-start
    best_test_loss = min(best_test_loss, epoch_test_loss)
    print('{}/{} ({:.2f}s - {:.2f}s remaining)'.format(epoch+1, n_epochs, time.time()-start, (n_epochs-epoch)*(elapsed/(epoch+1))))
    info = f'''Epoch: {epoch+1:03d}\tTrain Loss: {epoch_train_loss:.3f}\tTest: {epoch_test_loss:.3f}\tBest Test Loss: {best_test_loss:.4f}'''
    info += f'\nGender Accuracy: {val_gender_acc*100:.2f}%\tAge MAE: {val_age_mae:.2f}\n'
    print(info)

    val_gender_accuracies.append(val_gender_acc)
    val_age_maes.append(val_age_mae)

KeyboardInterrupt: 

In [ ]:
#Graph it

epochs = np.arange(1,(n_epochs+1))
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax = ax.flat
ax[0].plot(epochs, val_gender_accuracies, 'bo')
ax[1].plot(epochs, val_age_maes, 'r')
ax[0].set_xlabel('Epochs') ; ax[1].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy'); ax[1].set_ylabel('MAE')
ax[0].set_title('Validation Gender Accuracy')
ax[0].set_title('Validation Age Mean-Absolute-Error')
plt.show()